In [2]:
import pandas as pd
import pywhatkit
import keyboard
import time
from pytz import timezone
from datetime import datetime, timedelta, date

html_location = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSMYxGqQ46lMjzRew_mcIk-Y9tbnpbHEgzAg1yFRQlpzNaQctiwTzp0hoN2YQXvXYlEavkV23Q4_wqt/pub?output=xlsx"
todays_date = datetime.now(timezone("asia/kolkata")).date()
sheet_list = pd.ExcelFile(html_location).sheet_names
phone_list = pd.read_excel(html_location, sheet_name="Phone Number")
phone_list = dict(zip(phone_list["name"], phone_list["number"]))

print("Running for Date:- " + str(todays_date))
print("Teams Available: " + str(", ".join(sheet_list[:-1])))
print("list of people and numbers: " + str(phone_list) + "\n")

def send_message(df, phone_list, msg, send_col, sheet_name):
    if(df.shape[0] > 0):
        print(f"For {sheet_name}:-")
        for i in df.index:
            try:
                print("Sending the Message: " + str(msg + f" for Team {df['team'][i]}"))
                print(f"Sending To: {df[send_col][i]} {phone_list[df[send_col][i]]}")
                
                date = datetime.now(timezone('Asia/Kolkata')) + timedelta(minutes = 2)
                #pywhatkit.sendwhatmsg(f"+91{phone_list[df[send_col][i]]}", msg + f" for team {df['team'][i]}", date.hour, date.minute, 20, True, 5)
            except Exception as e:
                continue
    print("####################################\n")

def work_not_done(df, sheet_name):
    df = df[(df["date_diff"] <= 4) & ((df["completion_status"] == "NO")|(df["completion_status"].isna()))]
    df.reset_index(drop=True, inplace=True)
    msg = f"The {sheet_name} tasks has not been completed or marked completed"
    send_message(df, phone_list, msg, "completed_by", sheet_name)

def work_done_no_review(df, sheet_name):
    df = df[(df["completion_status"] == "YES") & ((df["review_status"] == "NO")|(df["review_status"].isna())) & (df["date_diff"] <= 1)]
    df.reset_index(drop=True, inplace=True)
    msg = f"The {sheet_name} tasks has been completed but not reviewed or marked reviewed"
    send_message(df, phone_list, msg, "review_by", sheet_name)

def assign_new(df, sheet_name):
    df = df[(df["completion_status"] == "YES") & (df["review_status"] == "YES")]
    df.reset_index(drop=True, inplace=True)
    msg = f"The {sheet_name} tasks has been completed and reviewed, its time to assign a new task"
    send_message(df, phone_list, msg, "review_by", sheet_name)


#Main function
for sheet_name in sheet_list[:-1]:
    main_df = pd.read_excel(html_location, sheet_name=sheet_name, usecols="A:F", skiprows=1)
    main_df.set_axis(["team","date","completed_by","completion_status","review_by","review_status"], axis=1, inplace=True)
    main_df.dropna(subset = ["team"], inplace=True)
    main_df['date'] = main_df['date'].astype('datetime64[ns]')
    main_df['date'] = [d.date() for d in main_df['date']]
    main_df["date_diff"] = main_df["date"] - todays_date
    main_df["date_diff"] = main_df["date_diff"].apply(str).str.split(' ').str[0].apply(int)
    
    print("Sending messages for work not done")
    work_not_done(main_df, sheet_name)
    print("Sending messages for review")
    work_done_no_review(main_df, sheet_name)
    print("Sending messages for assigning new task")
    assign_new(main_df, sheet_name)

Running for Date:- 2022-01-23
Teams Available: Social Media, Graphic Designing, Content Writing
list of people and numbers: {'sushmita': 9088185926, 'aarohi': 6394423076, 'mary': 8876504064, 'komal': 9536215043}

Sending messages for work not done
For Social Media:-
Sending the Message: The Social Media tasks has not been completed or marked completed for Team YOUTUBE
Sending To: komal 9536215043
####################################

Sending messages for review
For Social Media:-
Sending the Message: The Social Media tasks has been completed but not reviewed or marked reviewed for Team INSTAGRAM
Sending To: sushmita 9088185926
Sending the Message: The Social Media tasks has been completed but not reviewed or marked reviewed for Team LINKEDIN
Sending To: sushmita 9088185926
####################################

Sending messages for assigning new task
For Social Media:-
Sending the Message: The Social Media tasks has been completed and reviewed, its time to assign a new task for Team FAC